# Reacher - PPO

## Import packages

In [1]:
import sys
import random
from collections import namedtuple, deque

import numpy as np
import matplotlib.pyplot as plt
from unityagents import UnityEnvironment
from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from imported_utils import Batcher

In [2]:
DISCOUNT_RATE = 0.99
TAU = 0.95
GRADIENT_CLIP = 5
ROLLOUT_LENGTH = 2048
OPTIMIZATION_EPOCHS = 10
PPO_CLIP = 0.02
LOG_INTERVAL = 2048
MAX_STEPS = 2e7
MINI_BATCH_NUMBER = 32
ENTROPY_COEFICENT = 0.01
EPISODE_COUNT = 10
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

env = UnityEnvironment(file_name="Reacher")
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])


def layer_init(layer, w_scale=1.0):
    nn.init.orthogonal_(layer.weight.data)
    layer.weight.data.mul_(w_scale)
    nn.init.constant_(layer.bias.data, 0)
    return layer


class DummyBody(nn.Module):
    def __init__(self, state_dim):
        super(DummyBody, self).__init__()
        self.feature_dim = state_dim

    def forward(self, x):
        return x

class ActorCriticNet(nn.Module):
    def __init__(self, state_dim, action_dim, phi_body, actor_body, critic_body):
        super(ActorCriticNet, self).__init__()
        if phi_body is None: phi_body = DummyBody(state_dim)
        if actor_body is None: actor_body = DummyBody(phi_body.feature_dim)
        if critic_body is None: critic_body = DummyBody(phi_body.feature_dim)
        self.phi_body = phi_body
        self.actor_body = actor_body
        self.critic_body = critic_body
        self.fc_action = layer_init(nn.Linear(actor_body.feature_dim, action_dim), 1e-3)
        self.fc_critic = layer_init(nn.Linear(critic_body.feature_dim, 1), 1e-3)

        self.actor_params = list(self.actor_body.parameters()) + list(self.fc_action.parameters())
        self.critic_params = list(self.critic_body.parameters()) + list(self.fc_critic.parameters())
        self.phi_params = list(self.phi_body.parameters())


class GaussianActorCriticNet(nn.Module):
    def __init__(self,
                 state_dim,
                 action_dim,
                 phi_body=None,
                 actor_body=None,
                 critic_body=None):
        super(GaussianActorCriticNet, self).__init__()
        self.network = ActorCriticNet(state_dim, action_dim, phi_body, actor_body, critic_body)
        self.std = nn.Parameter(torch.ones(1, action_dim))
        self.to(DEVICE)

    def forward(self, obs, action=None):
        obs = torch.Tensor(obs)
        phi = self.network.phi_body(obs)
        phi_a = self.network.actor_body(phi)
        phi_v = self.network.critic_body(phi)
        mean = F.tanh(self.network.fc_action(phi_a))
        v = self.network.fc_critic(phi_v)
        dist = torch.distributions.Normal(mean, self.std)
        if action is None:
            action = dist.sample()
        log_prob = dist.log_prob(action)
        log_prob = torch.sum(log_prob, dim=1, keepdim=True)
        return action, log_prob, torch.Tensor(np.zeros((log_prob.size(0), 1))), v
    
class FCBody(nn.Module):
    def __init__(self, state_dim, hidden_units=(64, 64), gate=F.relu):
        super(FCBody, self).__init__()
        dims = (state_dim, ) + hidden_units
        self.layers = nn.ModuleList([layer_init(nn.Linear(dim_in, dim_out)) for dim_in, dim_out in zip(dims[:-1], dims[1:])])
        self.gate = gate
        self.feature_dim = dims[-1]

    def forward(self, x):
        for layer in self.layers:
            x = self.gate(layer(x))
        return x


policy = GaussianActorCriticNet(state_size, action_size, actor_body=FCBody(state_size), critic_body=FCBody(state_size))
optimizier = optim.Adam(policy.parameters(), 3e-4, eps=1e-5)


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


In [3]:

    
    
class PPOAgent(object):
    
    def __init__(self):
        self.network = policy
        self.opt = optimizier
        self.total_steps = 0
        self.all_rewards = np.zeros(num_agents)
        self.episode_rewards = []
        
        env_info = env.reset(train_mode=True)[brain_name]    
        self.states = env_info.vector_observations              

    def step(self):
        rollout = []
        env_info = env.reset(train_mode=True)[brain_name]    
        self.states = env_info.vector_observations  
        states = self.states
        for _ in range(ROLLOUT_LENGTH):
            actions, log_probs, _, values = self.network(states)
            env_info = env.step(actions.cpu().detach().numpy())[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            terminals = np.array([1 if t else 0 for t in env_info.local_done])
            self.all_rewards += rewards
            
            for i, terminal in enumerate(terminals):
                if terminals[i]:
                    self.episode_rewards.append(self.all_rewards[i])
                    self.all_rewards[i] = 0
                    
            rollout.append([states, values.detach(), actions.detach(), log_probs.detach(), rewards, 1 - terminals])
            states = next_states

        self.states = states
        pending_value = self.network(states)[-1]
        rollout.append([states, pending_value, None, None, None, None])

        processed_rollout = [None] * (len(rollout) - 1)
        advantages = torch.Tensor(np.zeros((num_agents, 1)))
        returns = pending_value.detach()
        for i in reversed(range(len(rollout) - 1)):
            states, value, actions, log_probs, rewards, terminals = rollout[i]
            terminals = torch.Tensor(terminals).unsqueeze(1)
            rewards = torch.Tensor(rewards).unsqueeze(1)
            actions = torch.Tensor(actions)
            states = torch.Tensor(states)
            next_value = rollout[i + 1][1]
            returns = rewards + DISCOUNT_RATE * terminals * returns

            td_error = rewards + DISCOUNT_RATE * terminals * next_value.detach() - value.detach()
            advantages = advantages * TAU * DISCOUNT_RATE * terminals + td_error
            processed_rollout[i] = [states, actions, log_probs, returns, advantages]

        states, actions, log_probs_old, returns, advantages = map(lambda x: torch.cat(x, dim=0), zip(*processed_rollout))
        advantages = (advantages - advantages.mean()) / advantages.std()

        batcher = Batcher(states.size(0) // MINI_BATCH_NUMBER, [np.arange(states.size(0))])
        for _ in range(OPTIMIZATION_EPOCHS):
            batcher.shuffle()
            while not batcher.end():
                batch_indices = batcher.next_batch()[0]
                batch_indices = torch.Tensor(batch_indices).long()
                sampled_states = states[batch_indices]
                sampled_actions = actions[batch_indices]
                sampled_log_probs_old = log_probs_old[batch_indices]
                sampled_returns = returns[batch_indices]
                sampled_advantages = advantages[batch_indices]

                _, log_probs, entropy_loss, values = self.network(sampled_states, sampled_actions)
                ratio = (log_probs - sampled_log_probs_old).exp()
                obj = ratio * sampled_advantages
                obj_clipped = ratio.clamp(1.0 - PPO_CLIP,
                                          1.0 + PPO_CLIP) * sampled_advantages
                policy_loss = -torch.min(obj, obj_clipped).mean(0) - ENTROPY_COEFICENT * entropy_loss.mean()

                value_loss = 0.5 * (sampled_returns - values).pow(2).mean()

                self.opt.zero_grad()
                (policy_loss + value_loss).backward()
                nn.utils.clip_grad_norm_(self.network.parameters(), GRADIENT_CLIP)
                self.opt.step()

        steps = ROLLOUT_LENGTH * num_agents
        self.total_steps += steps

In [4]:
agent = PPOAgent()
all_scores = []
for i in range(EPISODES_NUMBER):
    agent.step()
    
    env_info = env.reset(train_mode=True)[brain_name]    
    states = env_info.vector_observations                 
    scores = np.zeros(num_agents)                         
    while True:
        actions, log_probs, _, values = policy(states)
        env_info = env.step(actions.cpu().detach().numpy())[brain_name]
        next_states = env_info.vector_observations         
        rewards = env_info.rewards                         
        dones = env_info.local_done                     
        scores += env_info.rewards                      
        states = next_states                               
        if np.any(dones):                                  
            break
    m = np.mean(scores)
    all_scores.append(m)
    
    if len(all_scores) > 100:
        h = np.mean(np.array(all_scores[-100:]))
    else:
        h = np.mean(np.array(all_scores))
    if h > 30.0:
        agent.save(f"ppo-over-30-episode-{i}")
    print('Episode: {} Total score this episode: {} Average {}: {}'.format(i + 1, m, min(i + 1, 100), h))

Episode: 1 Total score this episode: 0.18999999575316906 Average 1: 0.18999999575316906
Episode: 2 Total score this episode: 0.09399999789893627 Average 2: 0.14199999682605266
Episode: 3 Total score this episode: 0.11999999731779099 Average 3: 0.1346666636566321
Episode: 4 Total score this episode: 0.24599999450147153 Average 4: 0.16249999636784196
Episode: 5 Total score this episode: 0.05649999873712659 Average 5: 0.1412999968416989
Episode: 6 Total score this episode: 0.1169999973848462 Average 6: 0.13724999693222345
Episode: 7 Total score this episode: 0.19499999564141035 Average 7: 0.1454999967478216
Episode: 8 Total score this episode: 0.24299999456852675 Average 8: 0.15768749647540972
Episode: 9 Total score this episode: 0.2699999939650297 Average 9: 0.17016666286314527
Episode: 10 Total score this episode: 0.2319999948143959 Average 10: 0.17634999605827034
